In [1]:
import requests
import json

from pathlib import Path
from pprint import pprint
from datetime import datetime

from openai import OpenAI
from langchain_openai import OpenAI, ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


In [2]:
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

## Import Prompt

In [23]:
with open("prompt_continuouseffects.txt", "r") as file:
    prompt_continuous_effects = file.read()
print(prompt_continuous_effects)

"""
You will get two cards from the cardgame Magic: the Gathering. {card1} and {card2} both have continuous effects.
You can assume that {card1} is played first, therefor has the earlier timestamp.
Use the {rules_db} as context for your decision.
You will get a {question} about two cards and have to make a decision.

You will follow these steps:

1. Give the oracle texts from both cards.
2. Determine the layer on which the effects of the cards are applied. Layers are defined in 613.1 in the {rules_db}.
3. Decide which rule is fitting in the case: Timestamp and Layers (613.1 to 613.7) or Dependency (613.8).
4. Give your judging: The continuous effect of which card is applied in game: {card1} or {card2}? Quote the effects.
5. Explain why you decided this way.
6. Quote the rule you used for your decision.

"""


## RAG
= Retrieved Augmented Generation
- ChatGPT's knowlegde of the rules is from January 2022: for current state the rulebook needs to be retrieved and passed to the LLM
- this data can provide context for the specific questions
### Loader
- simple loader to load .txt data: complete rule book from Magic: The Gathering

In [24]:
#loader = TextLoader("./data/rules_shortened.txt")
loader = TextLoader("./data/rules.txt")
rules_doc = loader.load()

### Splitter
- splitting the text into smaller chunks
- this text splitter splits text recursively by characters: ["\n\n", "\n", " ", ""], so first paragraphs, then sentences, then words
- keeps semantically related pieces together as long as possible

In [25]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
rules_splits = text_splitter.split_documents(rules_doc)
# print(rules_splits[0])

### Vector Store
- for storage of the embeddings
- embedding for capture semantic meaning of the text: that's how context is given for later question
- OpenAI embedding for ChatGPT
- Chroma as open-source embedding database
- finally, construct a retriever out of vectorstore 

In [26]:
rules_db = Chroma.from_documents(documents=rules_splits, embedding=OpenAIEmbeddings())
retriever_rules = rules_db.as_retriever()

- create new LLM: temperature 0 for more consistent, non-creative and therefor comparable output
- load template with prompt

In [27]:
# llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
model = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)


In [28]:
prompt_template = PromptTemplate.from_template(prompt_continuous_effects)
#prompt_template = ChatPromptTemplate.from_template(prompt_continuous_effects)

## Case 1: Blood Moon vs. Urborg, Tomb of Yawgmoth
- Get card information via scryfall.com and delete unnecessary data: 
- keep only: name, mana cost, converted mana cost, type, oracle text, colors, keywords

In [29]:
card1 = "Blood Moon"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card1} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card1 = json.loads(clean_json)
    with open('./data/card1.json', 'w') as f:
        json.dump(card1, f)
    print(card1)


{'name': 'Blood Moon', 'mana_cost': '{2}{R}', 'cmc': 3.0, 'type_line': 'Enchantment', 'oracle_text': 'Nonbasic lands are Mountains.', 'colors': ['R'], 'keywords': []}


In [30]:
card2 = "Urborg, Tomb of Yawgmoth"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card2} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    print(card2)
    print(card2)
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card2 = json.loads(clean_json)
    with open('./data/card2.json', 'w') as f:
        json.dump(card2, f)
    print(card2)

Urborg, Tomb of Yawgmoth
Urborg, Tomb of Yawgmoth
{'name': 'Urborg, Tomb of Yawgmoth', 'mana_cost': '', 'cmc': 0.0, 'type_line': 'Legendary Land', 'oracle_text': 'Each land is a Swamp in addition to its other land types.', 'colors': [], 'keywords': [], 'produced_mana': ['B']}


In [31]:
# testing Prompt with input_variables as json
prompt_template = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

- Use Chatmodel instead of LLM: more possible token in prompt
- Outputparser to parse message response from Chatmodel to str
- check testcases.md to compare expected outcome with ChatGPT's answer

In [32]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
answer = chain.invoke("What is the outcome of two played cards with continuous effects from Magic: the Gathering?")
print(answer)

1. Oracle texts:
- Blood Moon: "Nonbasic lands are Mountains."
- Urborg, Tomb of Yawgmoth: "Each land is a Swamp in addition to its other land types."

2. The effects of these cards are applied in Layer 4: Type-changing effects. This layer deals with effects that change the characteristics of an object, such as its types (land, creature, etc.).

3. In this case, we will use the Timestamp and Layers rule (613.1 to 613.7) to determine which continuous effect takes precedence.

4. Judging: The continuous effect of Blood Moon, which turns nonbasic lands into Mountains, will be applied in the game.

5. I decided this way because the Timestamp and Layers rule states that when two continuous effects are in conflict, the one with the later timestamp takes precedence. Since Blood Moon was played first, its effect will be applied over Urborg, Tomb of Yawgmoth.

6. Rule used: Timestamp and Layers (613.1 to 613.7)


- saving the answers as .txt for comparison by date & time: 

In [156]:
current_datetime = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))
file_name = current_datetime+".txt"

with open(f'./data/answers/{file_name}', 'w') as f:
    f.write(answer)

In [22]:
# cleanup
rules_db.delete_collection()

## Case 2a: Opalescence vs. Humility
- timestamp rule is determine the outcome: *Humility* is played last and it's effect overrides *Opalescence*

In [33]:
card1 = "Opalescence"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card1} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card1 = json.loads(clean_json)
    with open('./data/card1.json', 'w') as f:
        json.dump(card1, f)
    print(card1)

{'name': 'Opalescence', 'mana_cost': '{2}{W}{W}', 'cmc': 4.0, 'type_line': 'Enchantment', 'oracle_text': 'Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value.', 'colors': ['W'], 'keywords': []}


In [34]:
card2 = "Humility"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card2} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    print(card2)
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card2 = json.loads(clean_json)
    with open('./data/card2.json', 'w') as f:
        json.dump(card2, f)
    print(card2)

{'name': 'Opalescence', 'mana_cost': '{2}{W}{W}', 'cmc': 4.0, 'type_line': 'Enchantment', 'oracle_text': 'Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value.', 'colors': ['W'], 'keywords': []}
{'name': 'Humility', 'mana_cost': '{2}{W}{W}', 'cmc': 4.0, 'type_line': 'Enchantment', 'oracle_text': 'All creatures lose all abilities and have base power and toughness 1/1.', 'colors': ['W'], 'keywords': []}


In [35]:
prompt_template = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

In [36]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
answer = chain.invoke("What is the outcome of two played cards with continuous effects from Magic: the Gathering?")
print(answer)

1. Oracle texts:
- Opalescence: Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value.
- Humility: All creatures lose all abilities and have base power and toughness 1/1.

2. The effects of these cards are applied in Layer 4 (Type-changing effects) and Layer 6 (Ability-removing effects) according to the layer system in Magic: the Gathering.

3. In this case, we will use the Timestamp and Layers rule (613.1 to 613.7) to determine which continuous effect takes precedence.

4. Judging: The continuous effect of Opalescence is applied in the game. The effect is: Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value.

5. Explanation: According to the Timestamp and Layers rule, since Opalescence was played first and its effect falls in Layer 4 (Type-changing effects), it will be applied before Humility's effect in L

In [37]:
file_name = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))
# file_name = current_datetime+".txt"

with open(f'./data/answers/case2_{file_name}.txt', 'w') as f:
    f.write(answer)

## Case 2b: Humility vs. Opalescence 
- because timestamp is important here, testing the case with changed order

In [38]:
card1 = "Humility"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card1} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card1 = json.loads(clean_json)
    with open('./data/card1.json', 'w') as f:
        json.dump(card1, f)
    print(card1)

{'name': 'Humility', 'mana_cost': '{2}{W}{W}', 'cmc': 4.0, 'type_line': 'Enchantment', 'oracle_text': 'All creatures lose all abilities and have base power and toughness 1/1.', 'colors': ['W'], 'keywords': []}


In [40]:
card2 = "Opalescence"
response = requests.get(f'https://api.scryfall.com/cards/named?exact={card2} ', 
                 headers={'Accept': 'application/json'})
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary to delete keywords
    data = response.json()
    print(card2)
    keywords = ["object", "id", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
    for keyword in keywords:
        if keyword in data:
            del data[keyword]
    
    clean_json = json.dumps(data)
    # convert into json dict & save:
    card2 = json.loads(clean_json)
    with open('./data/card2.json', 'w') as f:
        json.dump(card2, f)
    print(card2)

Opalescence
{'name': 'Opalescence', 'mana_cost': '{2}{W}{W}', 'cmc': 4.0, 'type_line': 'Enchantment', 'oracle_text': 'Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value.', 'colors': ['W'], 'keywords': []}


In [41]:
prompt_template = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

In [42]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
answer = chain.invoke("What is the outcome of two played cards with continuous effects from Magic: the Gathering?")
print(answer)

1. Oracle texts:
- Humility: "All creatures lose all abilities and have base power and toughness 1/1."
- Opalescence: "Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value."

2. The effects of these cards are applied in different layers. The layer system in Magic: the Gathering determines the order in which continuous effects are applied to objects. In this case, Humility's effect affects the characteristics of creatures in layer 6 (Ability Adding or Removing Effects), while Opalescence's effect affects the type-changing characteristic of enchantments in layer 4 (Type-Changing Effects).

3. The rule that is fitting in this case is the Layer system (613.1 to 613.7).

4. Judging: The continuous effect of Humility is applied in the game. The effect of Humility overrides the effect of Opalescence.

5. Explanation: According to the layer system, effects are applied in a specific order, and effects in 

In [43]:
file_name = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))
# file_name = current_datetime+".txt"

with open(f'./data/answers/case2_{file_name}.txt', 'w') as f:
    f.write(answer)

In [ ]:
# cleanup
rules_db.delete_collection()